In [1]:
#Librerias

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import a_funciones as funciones

### Lectura de datos para modelado

In [2]:
df_models = pd.read_csv('data//df_employees_models.csv')
df_models.head()

,EmployeeID,Year,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,JobLevel,MaritalStatus,...,StockOptionLevel,TrainingTimesLastYear,YearsSinceLastPromotion,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,retirementType,resignationReason,Attrition
0,1,2015,51,Travel_Rarely,Sales,6,2,Life Sciences,1,Married,...,0,6,0,3,4,2,3,Not Applicable,Not Applicable,0
1,2,2015,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Single,...,1,3,1,3,2,4,2,Resignation,Others,1
2,3,2015,32,Travel_Frequently,Research & Development,17,4,Other,4,Married,...,3,2,0,2,2,1,3,Not Applicable,Not Applicable,0
3,4,2015,38,Non-Travel,Research & Development,2,5,Life Sciences,3,Married,...,3,5,7,4,4,3,2,Not Applicable,Not Applicable,0
4,5,2015,32,Travel_Rarely,Research & Development,10,1,Medical,1,Single,...,2,2,0,4,1,3,3,Not Applicable,Not Applicable,0


In [3]:
df_models.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   EmployeeID               8820 non-null   int64 
 1   Year                     8820 non-null   int64 
 2   Age                      8820 non-null   int64 
 3   BusinessTravel           8820 non-null   object
 4   Department               8820 non-null   object
 5   DistanceFromHome         8820 non-null   int64 
 6   Education                8820 non-null   int64 
 7   EducationField           8820 non-null   object
 8   JobLevel                 8820 non-null   int64 
 9   MaritalStatus            8820 non-null   object
 10  MonthlyIncome            8820 non-null   int64 
 11  NumCompaniesWorked       8820 non-null   int64 
 12  PercentSalaryHike        8820 non-null   int64 
 13  StockOptionLevel         8820 non-null   int64 
 14  TrainingTimesLastYear    8820 non-null  

In [4]:
# Separación de variables categoricas
list_cat=['BusinessTravel','Department','EducationField','MaritalStatus','retirementType', 'resignationReason']

In [5]:
y_train = df_models[df_models['Year']==2015]['Attrition']
X_train = df_models[df_models['Year']==2015].drop('Attrition', axis=1)

In [6]:
y_train

0       0
1       1
2       0
3       0
4       0
       ..
4405    0
4406    0
4407    0
4408    0
4409    0
Name: Attrition, Length: 4410, dtype: int64

In [7]:
X_test = df_models[df_models['Year']==2016].drop('Attrition', axis=1)

In [10]:
X_test

,EmployeeID,Year,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,JobLevel,MaritalStatus,...,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,YearsSinceLastPromotion,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,retirementType,resignationReason
4410,1,2016,51,Travel_Rarely,Sales,6,2,Life Sciences,1,Married,...,11,0,6,0,3,4,2,3,Not Applicable,Not Applicable
4411,4412,2016,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Single,...,23,1,3,1,3,2,4,2,Not Applicable,Not Applicable
4412,3,2016,32,Travel_Frequently,Research & Development,17,4,Other,4,Married,...,15,3,2,0,2,2,1,3,Not Applicable,Not Applicable
4413,4,2016,38,Non-Travel,Research & Development,2,5,Life Sciences,3,Married,...,11,3,5,7,4,4,3,2,Not Applicable,Not Applicable
4414,5,2016,32,Travel_Rarely,Research & Development,10,1,Medical,1,Single,...,12,2,2,0,4,1,3,3,Not Applicable,Not Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8815,4406,2016,42,Travel_Rarely,Research & Development,5,4,Medical,1,Single,...,17,1,5,0,4,1,3,3,Not Applicable,Not Applicable
8816,4407,2016,29,Travel_Rarely,Research & Development,2,4,Medical,1,Divorced,...,15,0,2,0,4,4,3,2,Not Applicable,Not Applicable
8817,4408,2016,25,Travel_Rarely,Research & Development,25,2,Life Sciences,2,Married,...,20,0,4,1,1,3,3,3,Not Applicable,Not Applicable
8818,4409,2016,42,Travel_Rarely,Sales,18,2,Medical,1,Divorced,...,14,1,2,7,4,1,3,2,Not Applicable,Not Applicable


In [8]:
# Formato dummies de train (2015)
X_train_dummies = pd.get_dummies(X_train, columns=list_cat, drop_first=True)

In [12]:
X_train_dummies

,EmployeeID,Year,Age,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,...,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,MaritalStatus_Married,MaritalStatus_Single,retirementType_Not Applicable,retirementType_Resignation,resignationReason_Others,resignationReason_Salary,resignationReason_Stress
0,1,2015,51,6,2,1,131160,1,11,0,...,False,False,False,True,False,True,False,False,False,False
1,2,2015,31,10,1,1,41890,0,23,1,...,False,False,False,False,True,False,True,True,False,False
2,3,2015,32,17,4,4,193280,1,15,3,...,False,True,False,True,False,True,False,False,False,False
3,4,2015,38,2,5,3,83210,3,11,3,...,False,False,False,True,False,True,False,False,False,False
4,5,2015,32,10,1,1,23420,4,12,2,...,True,False,False,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,2015,42,5,4,1,60290,3,17,1,...,True,False,False,False,True,True,False,False,False,False
4406,4407,2015,29,2,4,1,26790,2,15,0,...,True,False,False,False,False,True,False,False,False,False
4407,4408,2015,25,25,2,2,37020,0,20,0,...,False,False,False,True,False,True,False,False,False,False
4408,4409,2015,42,18,2,1,23980,0,14,1,...,True,False,False,False,False,True,False,False,False,False


In [9]:
# Convertir False a 0 y True a 1
X_train_dummies = X_train_dummies.replace({True: 1, False: 0})

In [10]:
X_train_dummies.head()

,EmployeeID,Year,Age,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,...,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,MaritalStatus_Married,MaritalStatus_Single,retirementType_Not Applicable,retirementType_Resignation,resignationReason_Others,resignationReason_Salary,resignationReason_Stress
0,1,2015,51,6,2,1,131160,1,11,0,...,0,0,0,1,0,1,0,0,0,0
1,2,2015,31,10,1,1,41890,0,23,1,...,0,0,0,0,1,0,1,1,0,0
2,3,2015,32,17,4,4,193280,1,15,3,...,0,1,0,1,0,1,0,0,0,0
3,4,2015,38,2,5,3,83210,3,11,3,...,0,0,0,1,0,1,0,0,0,0
4,5,2015,32,10,1,1,23420,4,12,2,...,1,0,0,0,1,1,0,0,0,0


In [11]:
X1= X_train_dummies.loc[:,~X_train_dummies.columns.isin(['EmployeeID'])]

In [12]:
# Escalado de datos de train (2015)
scaler=StandardScaler()
scaler.fit(X1)
X_std=scaler.transform(X1)
X_train_std=pd.DataFrame(X_std,columns=X1.columns)
X_train_std

,Year,Age,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,...,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,MaritalStatus_Married,MaritalStatus_Single,retirementType_Not Applicable,retirementType_Resignation,resignationReason_Others,resignationReason_Salary,resignationReason_Stress
0,0.0,1.541369,-0.393938,-0.891688,-0.961486,1.405136,-0.678464,-1.150554,-0.932014,2.483396,...,-0.679141,-0.243059,-0.314093,1.088232,-0.685565,0.415779,-0.391077,-0.268234,-0.202066,-0.164361
1,0.0,-0.648668,0.099639,-1.868426,-0.961486,-0.491661,-1.079486,2.129306,0.241988,0.155707,...,-0.679141,-0.243059,-0.314093,-0.918921,1.458650,-2.405123,2.557042,3.728089,-0.202066,-0.164361
2,0.0,-0.539166,0.963398,1.061787,1.749610,2.725053,-0.678464,-0.057267,2.589994,-0.620189,...,-0.679141,4.114223,-0.314093,1.088232,-0.685565,0.415779,-0.391077,-0.268234,-0.202066,-0.164361
3,0.0,0.117845,-0.887515,2.038524,0.845911,0.386301,0.123580,-1.150554,2.589994,1.707500,...,-0.679141,-0.243059,-0.314093,1.088232,-0.685565,0.415779,-0.391077,-0.268234,-0.202066,-0.164361
4,0.0,-0.539166,0.099639,-1.868426,-0.961486,-0.884109,0.524602,-0.877232,1.415991,-0.620189,...,1.472448,-0.243059,-0.314093,-0.918921,1.458650,0.415779,-0.391077,-0.268234,-0.202066,-0.164361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,0.0,0.555852,-0.517332,1.061787,-0.961486,-0.100700,0.123580,0.489376,0.241988,1.707500,...,1.472448,-0.243059,-0.314093,-0.918921,1.458650,0.415779,-0.391077,-0.268234,-0.202066,-0.164361
4406,0.0,-0.867672,-0.887515,1.061787,-0.961486,-0.812504,-0.277442,-0.057267,-0.932014,-0.620189,...,1.472448,-0.243059,-0.314093,-0.918921,-0.685565,0.415779,-0.391077,-0.268234,-0.202066,-0.164361
4407,0.0,-1.305679,1.950552,-0.891688,-0.057788,-0.595138,-1.079486,1.309341,-0.932014,0.931603,...,-0.679141,-0.243059,-0.314093,1.088232,-0.685565,0.415779,-0.391077,-0.268234,-0.202066,-0.164361
4408,0.0,0.555852,1.086793,-0.891688,-0.961486,-0.872210,-1.079486,-0.330589,0.241988,-0.620189,...,1.472448,-0.243059,-0.314093,-0.918921,-0.685565,0.415779,-0.391077,-0.268234,-0.202066,-0.164361


In [13]:
mcla = LogisticRegression()
mdtc= DecisionTreeClassifier()
mrfc= RandomForestClassifier()
mgbc=GradientBoostingClassifier()
modelos= [ mcla, mdtc, mrfc, mgbc]
var_names=funciones.sel_variables(modelos, X_train_std, y_train, threshold=0.07)
var_names.shape

(15,)

In [14]:
X2=X_train_std[var_names] ### matriz con variables seleccionadas
X2.info()
X_train_std.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Age                               4410 non-null   float64
 1   BusinessTravel_Travel_Frequently  4410 non-null   float64
 2   EducationField_Medical            4410 non-null   float64
 3   EducationField_Other              4410 non-null   float64
 4   EducationField_Technical Degree   4410 non-null   float64
 5   EnvironmentSatisfaction           4410 non-null   float64
 6   JobSatisfaction                   4410 non-null   float64
 7   MaritalStatus_Single              4410 non-null   float64
 8   MonthlyIncome                     4410 non-null   float64
 9   PercentSalaryHike                 4410 non-null   float64
 10  resignationReason_Others          4410 non-null   float64
 11  resignationReason_Salary          4410 non-null   float64
 12  resign

In [15]:
X2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Age                               4410 non-null   float64
 1   BusinessTravel_Travel_Frequently  4410 non-null   float64
 2   EducationField_Medical            4410 non-null   float64
 3   EducationField_Other              4410 non-null   float64
 4   EducationField_Technical Degree   4410 non-null   float64
 5   EnvironmentSatisfaction           4410 non-null   float64
 6   JobSatisfaction                   4410 non-null   float64
 7   MaritalStatus_Single              4410 non-null   float64
 8   MonthlyIncome                     4410 non-null   float64
 9   PercentSalaryHike                 4410 non-null   float64
 10  resignationReason_Others          4410 non-null   float64
 11  resignationReason_Salary          4410 non-null   float64
 12  resign

In [16]:
accuracy_df_original=funciones.medir_modelos(modelos,"accuracy",X_train_std,y_train,25) ## base con todas las variables 
accuracy_varsel=funciones.medir_modelos(modelos,"accuracy",X2,y_train,25) ### base con variables seleccionadas

In [17]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)
accuracy=pd.concat([accuracy_df_original,accuracy_varsel],axis=1)
accuracy

,reg_lineal,decision_tree,random_forest,gradient_boosting,reg_lineal,decision_tree,random_forest,gradient_boosting
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
9,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
